<a href="https://colab.research.google.com/github/ShaswataJash/Satlike/blob/main/Global_Optimization_in_SatLike.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download Weighted MaxSAT formulas

In [ ]:
!pip install gdown==3.6.4

In [ ]:
#downloading from my personal Google drive
import gdown
gdown.download('https://drive.google.com/uc?id=1MO34-v5jO2FlgDjyTjIpuMznOTkWYLY0', 'mse17-incomplete-weighted-benchmarks.zip', quiet=False) #418MB
gdown.download('https://drive.google.com/uc?id=1kBVV3VFQXFPyVnu4jmtQRJz6SH5PBuFC', 'ms18_incomplete_wt.zip', quiet=False) #780MB


In [ ]:
import shutil
import time

#unpacking of this zip file may take more than 30 seconds
start_time_of_unpacking = time.time()
shutil.unpack_archive('mse17-incomplete-weighted-benchmarks.zip', '.')
print("total time taken for unpacking = %s in seconds" % (time.time() - start_time_of_unpacking))

start_time_of_unpacking = time.time()
shutil.unpack_archive('ms18_incomplete_wt.zip', '.')
print("total time taken for unpacking = %s in seconds" % (time.time() - start_time_of_unpacking))

In [ ]:
import glob
import pprint
formula_files17 = glob.glob('/content/mse17-incomplete-weighted-benchmarks' + '/**/*.wcnf.gz', recursive=True)
#pprint.pprint(formula_files17)
print("total formula17 file count=%s" % (len(formula_files17)))
formula_files18 = glob.glob('/content/maxsat_instances/ms_evals/MS18/mse18-incomplete-weighted-benchmarks' + '/**/*.wcnf.gz', recursive=True)
#pprint.pprint(formula_files18)
print("total formula18 file count=%s" % (len(formula_files18)))

In [ ]:
import gdown
gdown.download("https://drive.google.com/uc?id=1OEFyZ9TErSWUr8dx5f5oMVL3shNyWKZU", '2017_results_incomplete_weighted_300s.html', quiet=False)
gdown.download("https://drive.google.com/uc?id=1SJ2IWED4Ob-H7emgSFgdPpb62avSOF_y", '2018_results_incomplete_weighted_300s.html', quiet=False)

In [ ]:
import pandas as pd

table_MN_2017 = pd.read_html('2017_results_incomplete_weighted_300s.html')
df_2017 = table_MN_2017[0]
df_2017.head()

In [ ]:
def df_table_clean_for_2017_res(x):
    if x.rfind('/') > 0:
        return x[x.rfind('/')+1:]
    else:
        return x

df_2017['Benchmark'] = df_2017['Benchmark'].map(df_table_clean_for_2017_res)
df_2017.head()

In [ ]:
difficult_problems_of_2017_benchmarks = []
for index, row in df_2017.iterrows():
    count_of_no_sol = 0
    for col in df_2017.columns:
        if row[col] == '0.0 (-)':
            count_of_no_sol += 1
    
    if count_of_no_sol >= 3:
        print(row['Benchmark'])
        difficult_problems_of_2017_benchmarks.append(row['Benchmark'])

print("num of difficult problem of 2017 benchmark =", len(difficult_problems_of_2017_benchmarks))

In [ ]:
import pandas as pd

table_MN_2018 = pd.read_html('2018_results_incomplete_weighted_300s.html')
df_2018 = table_MN_2018[0]
df_2018.head()

In [ ]:
difficult_problems_of_2018_benchmarks = []
for index, row in df_2018.iterrows():
    count_of_no_sol = 0
    for col in df_2018.columns:
        if row[col] == '0.0 (-)':
            count_of_no_sol += 1
    
    if count_of_no_sol >= 3:
        print(row['Benchmark'])
        difficult_problems_of_2018_benchmarks.append(row['Benchmark'])

print("num of difficult problem of 2018 benchmark =", len(difficult_problems_of_2018_benchmarks))

# Satlike Git code download

In [ ]:
!rm -Rf Satlike
!git clone https://github.com/ShaswataJash/Satlike.git
%cd /content/Satlike/

# SWIG installation for python interface of C++ Satlike

In [ ]:
!wget http://prdownloads.sourceforge.net/swig/swig-4.0.2.tar.gz

In [ ]:
!gunzip swig-4.0.2.tar.gz
!tar -xvf swig-4.0.2.tar

In [ ]:
%cd swig-4.0.2/
!./configure
!make
!make install

In [ ]:
%cd /content/Satlike/
!swig -c++ -python satlikew.i
!cat satlikew.i

In [ ]:
%%writefile satlikew.cpp

#include "basis_pms.h"
#include "pms.h"

In [ ]:
%%writefile setup.py

#!/usr/bin/env python

"""
setup.py file for satlikew
"""

from distutils.core import setup, Extension

#https://stackoverflow.com/questions/1676384/how-to-pass-flag-to-gcc-in-python-setup-py-script
satlikew_module = Extension('_satlikew',
                           sources=['satlikew_wrap.cxx', 'satlikew.cpp'],
                           #extra_compile_args=['-fsanitize=address -fsanitize=pointer-compare -fsanitize=pointer-subtract -fsanitize=leak -fsanitize=undefined']
                           #extra_link_args=['-fsanitize=address -fsanitize=pointer-compare -fsanitize=pointer-subtract -fsanitize=leak -fsanitize=undefined']
                           )

setup (name = 'satlikew',
       version = '0.1',
       author      = "Shaswata Jash",
       description = """To interact with SatLike3.0 algorithm from python""",
       ext_modules = [satlikew_module],
       py_modules = ["satlikew"],
       )

In [ ]:
#refer http://www.swig.org/Doc4.0/Python.html#Python_nn20
!python setup.py build_ext --inplace

In [ ]:
!ls -l /content/Satlike/_satlikew.cpython-37m-x86_64-linux-gnu.so

In [ ]:
!ldd /content/Satlike/_satlikew.cpython-37m-x86_64-linux-gnu.so

# Different scoring schemes

In [ ]:
class UnsatSatSeparateScoring:
    def __init__(self, s):
        self.satlike = s
        self.feasible_sol_found = False
        self.num_of_times_sat_sc = 0
        self.sat_sc_fract_sum = 0.0
        self.num_of_times_unsat_hc = 0
        self.unsat_hc_fract_sum = 0.0
        self.recent_score = 0.0

    def update_score(self, result):
        if (result >= 0):
            self.feasible_sol_found = True

        if (result >= 0):
            self.num_of_times_sat_sc += 1
            self.recent_score = (self.satlike.get_total_soft_weight() - result)/self.satlike.get_total_soft_weight()
            self.sat_sc_fract_sum += self.recent_score
        else:
            self.num_of_times_unsat_hc += 1
            self.recent_score = (result / self.satlike.get_num_hclauses()) #recall in this case, result itself will be -ve
            self.unsat_hc_fract_sum += self.recent_score

    def get_recent_score(self):
        #we are negating the final score because we have to minimize the score
        return -self.recent_score

    def get_final_score(self):
        #we are negating the final score because we have to minimize the score
        return -(self.sat_sc_fract_sum / self.num_of_times_sat_sc) if self.feasible_sol_found else -(self.unsat_hc_fract_sum / self.num_of_times_unsat_hc)

class OnlySatSoftScoring:
    def __init__(self, s):
        self.satlike = s
        self.iteration = 0
        self.sat_sc_fract_sum = 0
        self.recent_score = 0

    def update_score(self, result):
        self.recent_score = self.satlike.get_soft_unsat_weight_before_flip()/self.satlike.get_total_soft_weight()
        self.sat_sc_fract_sum += self.recent_score
        self.iteration += 1

    def get_recent_score(self):
        return self.recent_score

    def get_final_score(self):
        assert (SEARCH_PER_EPISODE-1) == self.iteration, "SEARCH_PER_EPISODE=%s self.iteration=%s" % (SEARCH_PER_EPISODE,self.iteration)
        return (self.sat_sc_fract_sum / self.iteration)

class SatSoftWithUnsatHWPenaltyScoringNormalized:
    def __init__(self, s):
        self.satlike = s
        self.iteration = 0
        self.sat_sc_fract_sum = 0
        self.unsat_hc_fract_sum = 0
        self.num_of_times_unsat_hc = 0
        self.recent_score = 0.0

    def update_score(self, result):
        self.recent_score = self.satlike.get_soft_unsat_weight_before_flip()/self.satlike.get_total_soft_weight()
        self.sat_sc_fract_sum += self.recent_score
        self.iteration += 1
        
        if(result < 0):
            hw_penalty = abs(result / self.satlike.get_num_hclauses()) #recall in this case, result itself will be -ve
            #self.recent_score += (1 + hw_penalty) 
            self.recent_score += hw_penalty 
            self.unsat_hc_fract_sum += hw_penalty
            self.num_of_times_unsat_hc += 1

    def get_recent_score(self):
        return self.recent_score

    def get_final_score(self):
        assert (SEARCH_PER_EPISODE-1) == self.iteration, "SEARCH_PER_EPISODE=%s self.iteration=%s" % (SEARCH_PER_EPISODE,self.iteration)
        return (self.sat_sc_fract_sum / self.iteration) + (self.unsat_hc_fract_sum / self.num_of_times_unsat_hc)


# decompressing formula files

In [ ]:
import os
import re
import gzip
import shutil

def decompress_formula_file(gzipped_file_name, VERBOSITY=0):
    
    work_dir = '/tmp/decompressed_formula'
    try: 
        os.mkdir(work_dir) 
    except OSError as error: 
        pass
      
    filename = os.path.split(gzipped_file_name)[-1]
    filename = re.sub(r"\.gz$", "", filename, flags=re.IGNORECASE)
    decompressed_filename = os.path.join(work_dir, filename)

    if VERBOSITY > 0: print("decompressed_filename = %s" % (decompressed_filename))
    #refer: https://stackoverflow.com/questions/55040442/how-to-register-gz-format-in-shutil-register-archive-format-to-use-same-format
    with gzip.open(gzipped_file_name, 'rb') as f_in: 
        with open(decompressed_filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    return decompressed_filename


# Example implementation of SATLike algorithm 

In [ ]:
import os
import sys
import satlikew
import time
import shutil
import re
import gzip
import math
from statistics import mean 
import matplotlib.pyplot as plt
import random
from collections import defaultdict
import pprint


def draw_plots(accumulated_scores,result_emitted_after, score_at_when_result_emitted, CHUNK_LEN):
    fig = plt.figure(figsize=(18,6))
    plt.plot([x for x in range(len(accumulated_scores))], accumulated_scores)
    plt.scatter(result_emitted_after, score_at_when_result_emitted, color='red')
            
    number_of_chunks = len(accumulated_scores) // CHUNK_LEN
    residual_chunk = True if len(accumulated_scores) % CHUNK_LEN > 0 else False
    if residual_chunk:
        number_of_chunks += 1
    prev_mean = None
    mean_diff = ''
    for i in range(number_of_chunks):
        start_of_chunk_index = i*CHUNK_LEN
        scores_in_chunk = accumulated_scores[start_of_chunk_index: (start_of_chunk_index+CHUNK_LEN)]
        avg_score_of_chunk = mean(scores_in_chunk)
        if prev_mean != None:
            mean_diff += ('%s ' % ((avg_score_of_chunk - prev_mean)/prev_mean))
        prev_mean = avg_score_of_chunk
        plt.plot([start_of_chunk_index,start_of_chunk_index+len(scores_in_chunk)],[avg_score_of_chunk,avg_score_of_chunk], color='magenta', linestyle="--")
    fig.show()

def cross_over_style1(my_satlike, rand_gen, VERBOSITY=0):
    different = 0
    for x in range(my_satlike.get_num_vars()):
        assert (my_satlike.get_init_sol(x+1) == 0) or (my_satlike.get_init_sol(x+1) == 1)
        if my_satlike.get_best_sol(x+1) != my_satlike.get_init_sol(x+1):
            different += 1
            p = rand_gen.uniform(0, 1)
            my_satlike.set_init_sol(x+1,
                my_satlike.get_best_sol(x+1) if p < 0.80 else (1 - my_satlike.get_best_sol(x+1)))
                
    if VERBOSITY > 0: print("init from crossover-style1 feasible_flag=", my_satlike.get_feasible_flag_state(), " different=", different)
    my_satlike.init_with_decimation_stepwise(False, True) #copy_best_sol = False, copy_init_sol = True

def cross_over_style2(my_satlike, rand_gen, VERBOSITY=0):
    for x in range(my_satlike.get_num_vars()):
        p = rand_gen.uniform(0, 1)
        my_satlike.set_init_sol(x+1,
            my_satlike.get_best_sol(x+1) if p < 0.10 else (1 - my_satlike.get_best_sol(x+1)))
                
    if VERBOSITY > 0: print("init from crossover-style2 feasible_flag=", my_satlike.get_feasible_flag_state())
    my_satlike.init_with_decimation_stepwise(False, True) #copy_best_sol = False, copy_init_sol = True

def cross_over_style3(my_satlike, rand_gen, flipped_var_stat, VERBOSITY=0):
    from_var_red_stat = 0
    for x in range(my_satlike.get_num_vars()):
        if (x+1) in flipped_var_stat:
            if (flipped_var_stat[x+1][0] < 0) or (flipped_var_stat[x+1][1] < 0):
                my_satlike.set_init_sol(x+1, 0 if flipped_var_stat[x+1][0] < flipped_var_stat[x+1][1] else 1)
                from_var_red_stat += 1
                continue
        p = rand_gen.uniform(0, 1)
        my_satlike.set_init_sol(x+1, 0 if p < 0.5 else 1)

    if VERBOSITY > 0: print("init from crossover-style3 feasible_flag=", my_satlike.get_feasible_flag_state() , " from_var_red_stat=", from_var_red_stat)
    my_satlike.init_with_decimation_stepwise(False, True) #copy_best_sol = False, copy_init_sol = True


def determine_var_to_score_reduction_stat (total_var_count, flipped_var_to_score_red_stat, VERBOSITY=0):

    if len(flipped_var_to_score_red_stat) == 0:
        return {}

    flipped_var_stat = {}
    for x in range(total_var_count):
        key = x+1

        if not (key in flipped_var_to_score_red_stat):
            if (VERBOSITY > 1): print("var=%s is never considered for flipping" % (key))
            continue
                
        val = flipped_var_to_score_red_stat[key]
        flipped_var_stat[key] = [0,0]

        if val[0][1] == 0:
            if (VERBOSITY > 1): print("var=%s for val=0 has never fliped into" % (key))
        else:
            flipped_var_stat[key][0] = val[0][0] / val[0][1]
            if (VERBOSITY > 1) and (flipped_var_stat[key][0] < 0):
                print("var=%s for val=0 avg_red=%s (sum=%s, count=%s)" % (key, flipped_var_stat[key][0], val[0][0], val[0][1]))

        if val[1][1] == 0:
            if (VERBOSITY > 1): print("var=%s for val=1 has never fliped into" % (key))
        else:
            flipped_var_stat[key][1] = val[1][0] / val[1][1]
            if (VERBOSITY > 1) and (flipped_var_stat[key][1] < 0):
                print("var=%s for val=1 avg_red=%s (sum=%s, count=%s)" % (key, flipped_var_stat[key][1], val[1][0], val[1][1]))

    return flipped_var_stat

def satlike_execution(decompressed_filename, seed, initial_max_flip = None, adaptive_search=True, max_non_improve_flip=None, 
                      ext_hd_count=None, ext_smooth_probab=None, ext_hinc=None, ext_softclause_weight=None, lower_unsat_weight_acheive_hook = None,
                      time_budget_s = 300, premptive_episode_closure=False, init_sol_from_crossover="", VERBOSITY = 0, draw_figure=False, plot_solution_if_new=False):

    CHUNK_LEN = 1000000
     
    satlike = satlikew.Satlike()
    if VERBOSITY > 0: print("satlike object created %s sec before" % (satlike.get_runtime()))
    try:

        satlike.build_instance(decompressed_filename)

        if initial_max_flip != None:
            satlike.set_initial_max_flip(initial_max_flip)
        satlike.algo_init(seed, 0, True, todebug=False) #RAND_GEN_TYPE rType=MINSTD=0
        if VERBOSITY > 0: print("formula num_of_var=%s num_of_hclause=%s num_of_sclause=%s top_clause_wt=%s total_soft_wt=%s"
                              % (satlike.get_num_vars(), satlike.get_num_hclauses(), satlike.get_num_sclauses(), 
                                 satlike.get_top_clause_weight(), satlike.get_total_soft_weight()))

        last_soft_unsat_weight = satlike.get_total_soft_weight()+1

        start_time = time.time()
        break_from_outer_loop = False
        iteration_count=0
        iteration_count_when_lowest_unsat_weight_acheived=-1
        improvement_iteration = 0

        flipped_var_to_score_red_stat = {}
        r = random.Random(seed)
        exploit_cycle = True
        changing_seed = seed

        if plot_solution_if_new:
            found_solution_count = defaultdict(int)

        while break_from_outer_loop == False:
            
            if (len(init_sol_from_crossover) > 0) and (last_soft_unsat_weight < (satlike.get_total_soft_weight()+1)):

                if init_sol_from_crossover == 'style1':
                    cross_over_style1(satlike, r, VERBOSITY)
                    
                elif init_sol_from_crossover == 'style2':
                    cross_over_style2(satlike, r, VERBOSITY)
                    
                elif init_sol_from_crossover == 'style3':
                    my_flipped_var_stat = determine_var_to_score_reduction_stat(satlike.get_num_vars(), flipped_var_to_score_red_stat, VERBOSITY)
                    cross_over_style3(satlike, r, my_flipped_var_stat, VERBOSITY)
                    
                elif init_sol_from_crossover == 'style4':
                    satlike.init_with_decimation_stepwise(True, False) #copy_best_sol = True, copy_init_sol = False

                elif init_sol_from_crossover == 'style5':
                    if exploit_cycle:
                        satlike.init_with_decimation_stepwise(True, False) #copy_best_sol = True, copy_init_sol = False
                    else:
                        changing_seed += 4096
                        satlike.algo_init(changing_seed, 0, True, todebug=False) #RAND_GEN_TYPE rType=MINSTD=0, as seeds are changing rang-generator has to be recreated
                        satlike.init_with_decimation_stepwise(False, False)
                    if VERBOSITY > 0: print("init exploit_cycle=", exploit_cycle, " current_seed=", changing_seed)
                               
            else:
                if VERBOSITY > 0: print("init from decimation feasible_flag=", satlike.get_feasible_flag_state())
                satlike.init_with_decimation_stepwise(False, False)
            
            current_step = 1
            scorer = SatSoftWithUnsatHWPenaltyScoringNormalized(satlike)

            if draw_figure or premptive_episode_closure:
                accumulated_scores = []
                result_emitted_after = []
                score_at_when_result_emitted = []
                if plot_solution_if_new:
                    starting_sol_bitstr = ''.join([str(satlike.get_current_sol(x+1)) for x in range(satlike.get_num_vars())])

            chunk_index = 0
            prev_mean = None

            flipped_var_in_prev_step = -1
            fliped_val_of_var = -1
            prev_score_in_last_flip = -1

            any_improvement_found_in_current_episode = False
            
            #for (unsigned int current_step = 1; current_step<get_max_flips(); ++current_step)
            #to keep the looping as original C++ code, condition check is done against (satlike.get_max_flips()-1)
            while current_step < (satlike.get_max_flips()-1):
                current_step += 1
                iteration_count += 1
                result = satlike.local_search_stepwise(
                    ext_hd_count          if (ext_hd_count != None) else satlike.get_hd_count_threshold(), 
                    ext_smooth_probab     if (ext_smooth_probab != None) else satlike.get_smooth_probability(), 
                    ext_hinc              if (ext_hinc != None) else satlike.get_h_inc(), 
                    ext_softclause_weight if (ext_softclause_weight != None) else satlike.get_softclause_weight_threshold(), 
                    max_non_improve_flip  if (max_non_improve_flip != None) else satlike.get_max_non_improve_flip(),
                    current_step,
                    adaptive_search,
                    0) #verbose_level=0

                scorer.update_score(result)
                current_score = scorer.get_recent_score()

                if init_sol_from_crossover == 'style3':
                    if flipped_var_in_prev_step != -1:
                        if not (flipped_var_in_prev_step in flipped_var_to_score_red_stat):
                            flipped_var_to_score_red_stat[flipped_var_in_prev_step] = {0:[0,0], 1:[0,0]} #list structure [sum_of_reduction_of_score,count]
                        flipped_var_to_score_red_stat[flipped_var_in_prev_step][fliped_val_of_var][0] += (prev_score_in_last_flip - current_score)
                        flipped_var_to_score_red_stat[flipped_var_in_prev_step][fliped_val_of_var][1] += 1
                    prev_score_in_last_flip = current_score

                    flipped_var_in_prev_step = satlike.get_flipped_var()
                    fliped_val_of_var = satlike.get_current_sol(flipped_var_in_prev_step)
                    assert (fliped_val_of_var == 0) or (fliped_val_of_var == 1)

                if draw_figure or premptive_episode_closure: accumulated_scores.append(current_score)
              
                if (result >= 0) :

                    if (result < last_soft_unsat_weight):
                        any_improvement_found_in_current_episode = True
                        improvement_iteration += 1
                        if VERBOSITY > 0: print("iteration_count=%s opt_unsat_weight = %s score=%s time-taken in sec=%s improvement_iteration=%s" % 
                            (iteration_count, result, current_score, time.time() - start_time, improvement_iteration), flush=True)
                        if lower_unsat_weight_acheive_hook != None:
                            working_sol = [satlike.get_best_sol(x+1) for x in range(satlike.get_num_vars())]
                            lower_unsat_weight_acheive_hook(result, working_sol, improvement_iteration, VERBOSITY)
                        last_soft_unsat_weight = result
                        iteration_count_when_lowest_unsat_weight_acheived = iteration_count

                    if draw_figure:
                        if plot_solution_if_new:
                            if not (starting_sol_bitstr in found_solution_count):
                                result_emitted_after.append(len(accumulated_scores)-1)
                                score_at_when_result_emitted.append(current_score)
                            found_solution_count[starting_sol_bitstr] += 1 
                        else:
                            result_emitted_after.append(len(accumulated_scores)-1)
                            score_at_when_result_emitted.append(current_score)

                    if last_soft_unsat_weight == 0:
                        break_from_outer_loop = True
                        break
            
                if time.time() - start_time > time_budget_s:
                    if VERBOSITY > 0: print("Experiment time (%s) exhausted, iteration_count=%s" % (time_budget_s, iteration_count), flush=True)
                    break_from_outer_loop = True
                    break

                if premptive_episode_closure and (len(accumulated_scores) % CHUNK_LEN == 0):
                    start_of_chunk_index = chunk_index*CHUNK_LEN
                    scores_in_chunk = accumulated_scores[start_of_chunk_index: (start_of_chunk_index+CHUNK_LEN)]
                    avg_score_of_chunk = mean(scores_in_chunk)
                    if prev_mean != None:
                        fract_diff = ((avg_score_of_chunk - prev_mean)/prev_mean)
                        percent_diff = fract_diff * 100.0
                        if VERBOSITY > 0: print("percent_diff=%s avg_score_of_chunk=%s prev_mean=%s iteration_count=%s" 
                                  % (percent_diff, avg_score_of_chunk, prev_mean, iteration_count), flush=True)
                        if math.floor(abs(percent_diff)) <= 1.0:
                            break

                    prev_mean = avg_score_of_chunk
                    chunk_index += 1

                if plot_solution_if_new:
                    flipped_var_in_prev_step = satlike.get_flipped_var()
                    fliped_val_of_var = satlike.get_current_sol(flipped_var_in_prev_step)
                    assert (fliped_val_of_var == 0) or (fliped_val_of_var == 1)
                    
                    index_to_replace = flipped_var_in_prev_step-1
                    char_to_replace = '1' if starting_sol_bitstr[flipped_var_in_prev_step-1] == '0' else '0'
                    starting_sol_bitstr = starting_sol_bitstr[:index_to_replace] + char_to_replace + starting_sol_bitstr[index_to_replace+1:]
                    assert starting_sol_bitstr[flipped_var_in_prev_step-1] == str(fliped_val_of_var)

            
            if draw_figure: draw_plots(accumulated_scores, result_emitted_after, score_at_when_result_emitted,CHUNK_LEN)
            if VERBOSITY > 0: print("Episode completed, iteration_count=%s time-taken in sec=%s" % (iteration_count, time.time() - start_time), flush=True)
            
            if (init_sol_from_crossover == 'style5') and (last_soft_unsat_weight < (satlike.get_total_soft_weight()+1)):
                exploit_cycle = False if (False == any_improvement_found_in_current_episode) else True

    except Exception:
        ex_type = sys.exc_info()[0]
        ex_value= sys.exc_info()[1]
        print("Exception type: %s Exception message: %s" %(ex_type.__name__,ex_value),flush=True)
    finally:
        satlike.free_memory()

    del satlike

    return iteration_count_when_lowest_unsat_weight_acheived,last_soft_unsat_weight


In [ ]:
formula_for_experiment1 = '/content/mse17-incomplete-weighted-benchmarks/correlation-clustering-Rounded_CorrelationClustering_Vowel_BINARY_N760_D0.200.wcnf.gz'
formula_for_experiment2 = '/content/mse17-incomplete-weighted-benchmarks/rna-alignment-k100-42-56.rna.pre.wcnf.gz'
#formula_for_experiment3 = '/content/mse17-incomplete-weighted-benchmarks/causal-discovery-causal_carpo_8_1000.wcnf.gz'
#formula_for_experiment4 = '/content/mse17-incomplete-weighted-benchmarks/correlation-clustering-Rounded_CorrelationClustering_Ecoli_BINARY_N280_D0.200.wcnf.gz'
 
result_for_experiment1_original = satlike_execution(decompress_formula_file(formula_for_experiment1), 1, VERBOSITY=1)
assert result_for_experiment1_original[0] == 18719817 #iteration
assert result_for_experiment1_original[1] == 91166379 #lowest_unsat_weight

result_for_experiment2_original = satlike_execution(decompress_formula_file(formula_for_experiment2), 1, VERBOSITY=1)
assert result_for_experiment2_original[0] == 35984 #iteration
assert result_for_experiment2_original[1] == 1847  #lowest_unsat_weight

In [ ]:

formula_for_experiment1 = '/content/mse17-incomplete-weighted-benchmarks/correlation-clustering-Rounded_CorrelationClustering_Vowel_BINARY_N760_D0.200.wcnf.gz'
formula_for_experiment2 = '/content/mse17-incomplete-weighted-benchmarks/rna-alignment-k100-42-56.rna.pre.wcnf.gz'
formula_for_experiment3 = '/content/mse17-incomplete-weighted-benchmarks/causal-discovery-causal_carpo_8_1000.wcnf.gz'
#formula_for_experiment = '/content/mse17-incomplete-weighted-benchmarks/correlation-clustering-Rounded_CorrelationClustering_Ecoli_BINARY_N280_D0.200.wcnf.gz'

 
result_for_experiment1_with_premptive_episode = satlike_execution(decompress_formula_file(formula_for_experiment1), 1, premptive_episode_closure=True,
                                                              init_sol_from_crossover='style5', VERBOSITY=1, draw_figure=True)

'''
assert result_for_experiment1_with_premptive_episode[0] == 9088683 #iteration
assert result_for_experiment1_with_premptive_episode[1] == 93351646 #lowest_unsat_weight

result_for_experiment2_with_premptive_episode = satlike_execution(decompress_formula_file(formula_for_experiment2), 1, premptive_episode_closure=True,
                                                              init_sol_from_crossover='style5', time_budget_s=600, VERBOSITY=1, draw_figure=True)
#assert result_for_experiment2_with_premptive_episode[0] == 8002246 #iteration
#assert result_for_experiment2_with_premptive_episode[1] == 1843 #lowest_unsat_weight


result_for_experiment3_with_premptive_episode = satlike_execution(decompress_formula_file(formula_for_experiment3), 1, premptive_episode_closure=True,
                                                              init_sol_from_crossover='style5', time_budget_s=900, VERBOSITY=1, draw_figure=True)

'''

# Ray-tune experiment

In [ ]:
!whoami
!pwd
!python -V
!pip install ray[tune]

In [ ]:
!rm -Rf /tmp/decompressed_formula
!mkdir /tmp/decompressed_formula
#!ray start --help
#Without explicit information about binding dashboard-host to 127.0.0.1, dashboard can't be connected in google-colab
!ray start --head --port=6379 --object-manager-port=8076 --include-dashboard True --dashboard-host 127.0.0.1 --dashboard-port 8265 &

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8265)"))

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ~/ray_results

In [ ]:
CHOSEN_METRIC = 'episode_unsat_weight'

In [ ]:
import ray
from ray import tune
import satlikew
import time
import random

class GlobalOptimInSATLike(tune.Trainable):
    def setup(self, config):
        self.start_time = time.time()
        if config['verbosity'] > 0: print("object-id=%s setup with config=%s" % (id(self),config))
        self.config = config
        self.satlike = satlikew.Satlike()
        self.satlike.build_instance(self.config['decomp_file'])
        self.changing_seed = (int)(config['random_seed'])
        self.satlike.algo_init(self.changing_seed, 0, True, todebug=False) #RAND_GEN_TYPE rType=MINSTD=0
        self.last_soft_unsat_weight = self.satlike.get_total_soft_weight()+1
        self.iterations = 0
        self.exploit_cycle=True
        self.global_algo_cont = True
        self.__reset_episode(False)
        
    def __reset_episode(self, override):
        if override:
            if self.config['verbosity'] > 0: print("init overide")
            self.satlike.init_with_decimation_stepwise(False, True) #copy_best_sol = False, copy_init_sol = True
        elif (self.last_soft_unsat_weight < (self.satlike.get_total_soft_weight()+1)):
            if self.exploit_cycle:
                self.satlike.init_with_decimation_stepwise(True, False) #copy_best_sol = True, copy_init_sol = False
            else:
                self.changing_seed += 4096
                self.satlike.algo_init(self.changing_seed, 0, True, todebug=False) #RAND_GEN_TYPE rType=MINSTD=0, as seeds are changing rang-generator has to be recreated
                self.satlike.init_with_decimation_stepwise(False, False)
            if self.config['verbosity'] > 0: print("init exploit_cycle=", self.exploit_cycle, " current_seed=", self.changing_seed)
        else:
            if self.config['verbosity'] > 0: print("init from decimation feasible_flag=", self.satlike.get_feasible_flag_state())
            self.satlike.init_with_decimation_stepwise(False, False)
        
        self.current_step = 1
        self.scorer = SatSoftWithUnsatHWPenaltyScoringNormalized(self.satlike)

        self.accumulated_scores = []
        self.chunk_index = 0
        self.prev_mean = None

        self.any_improvement_found_in_current_episode = False

    def step(self):  # This is called iteratively.

        improvement_found_in_current_chunk = False
        
        while self.current_step < self.satlike.get_max_flips() and self.global_algo_cont:
            self.current_step += 1
            self.iterations += 1
            result = self.satlike.local_search_stepwise(
                (int)(self.config['hd_count_threshold']), 
                self.config['smooth_probability'], 
                (int)(self.config['h_inc']), 
                (int)(self.config['softclause_weight_threshold']), 
                self.satlike.get_max_non_improve_flip(),
                self.current_step,
                True, #adaptive_search=True
                0) #verbose_level=0

            self.scorer.update_score(result)
            current_score = self.scorer.get_recent_score()
            if self.config['premptive_episode_closure']: self.accumulated_scores.append(current_score)
            
            if (result >= 0) and (result < self.last_soft_unsat_weight):
                if self.config['verbosity'] > 0: print("iterations=%s unsat_soft=%s score=%s time=%s" 
                                                % (self.iterations, result, current_score, time.time() - self.start_time))
                self.last_soft_unsat_weight = result
                improvement_found_in_current_chunk = True
                self.any_improvement_found_in_current_episode = True
                if self.last_soft_unsat_weight == 0:
                    self.global_algo_cont = False

            if self.config['premptive_episode_closure'] and (len(self.accumulated_scores) % self.config['CHUNK_LEN'] == 0):
                start_of_chunk_index = self.chunk_index*self.config['CHUNK_LEN']
                scores_in_chunk = self.accumulated_scores[start_of_chunk_index: (start_of_chunk_index+self.config['CHUNK_LEN'])]
                avg_score_of_chunk = mean(scores_in_chunk)
                if self.prev_mean != None:
                    fract_diff = ((avg_score_of_chunk - self.prev_mean)/self.prev_mean)
                    percent_diff = fract_diff * 100.0
                    if self.config['verbosity'] > 0: print("percent_diff=%s avg_score_of_chunk=%s prev_mean=%s iteration_count=%s" 
                                % (percent_diff, avg_score_of_chunk, self.prev_mean, self.iterations), flush=True)
                    if math.floor(abs(percent_diff)) <= 1.0:
                        self.__reset_episode(False)
                        break

                self.prev_mean = avg_score_of_chunk
                self.chunk_index += 1

            if (self.iterations % self.config['CHUNK_LEN']) == 0:
                break #on every self.config['CHUNK_LEN'], emit some result so that pertubation logic can be applied frequently

        if (self.last_soft_unsat_weight < (self.satlike.get_total_soft_weight()+1)) and (self.current_step >= self.satlike.get_max_flips()):
            self.exploit_cycle = False if (False == self.any_improvement_found_in_current_episode) else True
            self.__reset_episode(False)

        return {CHOSEN_METRIC: self.last_soft_unsat_weight, 
                "done": True if self.last_soft_unsat_weight == 0 else False,
                "should_checkpoint": improvement_found_in_current_chunk}

    def save_checkpoint(self, tmp_checkpoint_dir):
        working_sol = []
        if self.last_soft_unsat_weight < (self.satlike.get_total_soft_weight()+1):
            working_sol = [self.satlike.get_best_sol(x+1) for x in range(self.satlike.get_num_vars())]
        return {CHOSEN_METRIC: self.last_soft_unsat_weight, "solution": working_sol}

    def load_checkpoint(self, checkpointed_data):
        if len(checkpointed_data['solution']) == self.satlike.get_num_vars():
            if self.config['verbosity'] > 0: print("Recevied checkpoint from metric=", checkpointed_data[CHOSEN_METRIC], " local last_soft_unsat_weight=", self.last_soft_unsat_weight)
            for x in range(self.satlike.get_num_vars()):
                self.satlike.set_init_sol(x+1,checkpointed_data['solution'][x])
            self.__reset_episode(True)

    def reset_config(self, new_config):
        if self.config['verbosity'] > 0: print("old config=%s new config=%s" % (self.config, new_config))
        self.config = new_config
        return True

    def stop(self):
        """Releases all resources used by this trainable.

        Calls ``Trainable.cleanup`` internally. Subclasses should override
        ``Trainable.cleanup`` for custom cleanup procedures.
        """
        self.global_algo_cont = False #so that currently ongoing step should exit as quickly as possible

        self._result_logger.flush()
        self._result_logger.close()
        if self._monitor.is_alive():
            self._monitor.stop()
            self._monitor.join()
        self.cleanup()

        self._close_logfiles()

    def cleanup(self):
        self.satlike.free_memory()
        del self.satlike

In [ ]:
from ray.tune.schedulers import PopulationBasedTraining
import sys

def conduct_tune_experiment_with_PBT(total_exp_time=1200):
    ray.shutdown()
    ray.init(address='auto', ignore_reinit_error=True)
    print('''This cluster consists of {} nodes in total {} CPU resources in total '''.format(len(ray.nodes()), ray.cluster_resources()['CPU']))

    #formula_for_experiment = '/content/mse17-incomplete-weighted-benchmarks/correlation-clustering-Rounded_CorrelationClustering_Vowel_BINARY_N760_D0.200.wcnf.gz'
    #formula_for_experiment = '/content/mse17-incomplete-weighted-benchmarks/rna-alignment-k100-42-56.rna.pre.wcnf.gz'
    formula_for_experiment = '/content/mse17-incomplete-weighted-benchmarks/causal-discovery-causal_carpo_8_1000.wcnf.gz'
    #formula_for_experiment = '/content/mse17-incomplete-weighted-benchmarks/correlation-clustering-Rounded_CorrelationClustering_Ecoli_BINARY_N280_D0.200.wcnf.gz'

    decompressed_file = decompress_formula_file(formula_for_experiment)

    pbt = PopulationBasedTraining(
        time_attr="training_iteration",
        perturbation_interval=4,

        #time_attr='time_total_s',
        #perturbation_interval=60.0,
        
        hyperparam_mutations={
            "hd_count_threshold": tune.uniform(1 , 100), #hd_count_threshold, default 15
            "smooth_probability": tune.uniform(0.0000001, 0.1), #smooth_probability, default either .01 or .0000001
            "h_inc": tune.uniform(1,1000),  #h_inc, default 3 or 300
            "softclause_weight_threshold" : tune.uniform(0, 2000), #softclause_weight_threshold,  default 0 or 500
        },
        
        synch = False)
    
    satlike = satlikew.Satlike()
    satlike.build_instance(decompressed_file)
    satlike.algo_init(1, 0, True, todebug=False)
    ext_hd_count = satlike.get_hd_count_threshold() 
    ext_smooth_probab = satlike.get_smooth_probability()
    ext_hinc = satlike.get_h_inc()
    ext_softclause_weight = satlike.get_softclause_weight_threshold()
    print('ext_hd_count=%s ext_smooth_probab=%s ext_hinc=%s ext_softclause_weight=%s' % 
          (ext_hd_count,ext_smooth_probab, ext_hinc, ext_softclause_weight))
    satlike.free_memory()
    del satlike
    
    NUM_OF_PARALLEL_SAMPLE = (int)(ray.cluster_resources()['CPU'])

    try:
        analysis = tune.run(
            GlobalOptimInSATLike,
            name="GLOBAL-OPTIM-IN-SatLike-Using-PBT",
            resources_per_trial={
                "cpu": 1, 
            },
            metric = CHOSEN_METRIC,
            mode = 'min',
            time_budget_s=total_exp_time,
            #num_samples=NUM_OF_PARALLEL_SAMPLE,
            num_samples=1,
            scheduler=pbt,
            reuse_actors=True, #so that satlike current states are not destroyed
            config={
                "hd_count_threshold": ext_hd_count,
                "smooth_probability": ext_smooth_probab, 
                "h_inc": ext_hinc,  
                "softclause_weight_threshold" : ext_softclause_weight,
                "decomp_file": decompressed_file,
                "cross_over_sampling_probability": 0.5,
                "verbosity": 1,
                "premptive_episode_closure": True,
                'CHUNK_LEN': 1000000,
                "random_seed": tune.grid_search([x*123 for x in range(NUM_OF_PARALLEL_SAMPLE)])
            },
            fail_fast=True,
            #progress_reporter = TrialTerminationReporter((int)(ray.cluster_resources()['CPU']))
        )

        print("Best config: %s best_result=%s" % (analysis.best_config, analysis.best_result))
    except:
        print("Oops! ", sys.exc_info()[0], " occurred.")

    ray.shutdown()



In [ ]:
conduct_tune_experiment_with_PBT()

In [ ]:
!g++ -O2 -Wall -g -fstack-protector-strong -D_FORTIFY_SOURCE=2 pms.cpp -o pms.out
!ldd pms.out

In [ ]:
#!./pms.out -c -r 1 -v 2 -h 814 -t 12 -s 0.00704856 -e 663 -z 1000000  "/tmp/causal-discovery-causal_carpo_8_1000.wcnf"
#!./pms.out -c -r 1 -v 2 -h 706 -t 18 -s 0.0032607 -e 205  "/tmp/causal-discovery-causal_carpo_8_1000.wcnf" #opt_unsat_weight=4505953
#!./pms.out -c -r 1 -v 2  "/tmp/causal-discovery-causal_carpo_8_1000.wcnf"	#soft_unsat_weight=4937192

#!./pms.out -c -r 1 -v 1  -t 10  -s 0.011881809087992991  -h 45 -e 608 "/tmp/correlation-clustering-Rounded_CorrelationClustering_Ecoli_BINARY_N280_D0.200.wcnf" #soft_unsat_weight=33568165
#!./pms.out -c -r 1 -v 1  -t 26  -s 0.041293033872572715  -h 38 -e 1253 "/tmp/correlation-clustering-Rounded_CorrelationClustering_Ecoli_BINARY_N280_D0.200.wcnf"

!./pms.out -c -r 1 -v 1 -t 25 -s 0.06132548484136097 -h 1 -e 938 "/tmp/correlation-clustering-Rounded_CorrelationClustering_Vowel_BINARY_N760_D0.200.wcnf"